In [ ]:
import numpy as np
from scipy.stats import ortho_group
import sklearn.linear_model as lin
import time
from mpc import *
import matplotlib.pyplot as plt

# Worse instance

In [ ]:
def subspace_basis(t, d):
    U = ortho_group.rvs(d)
    return U[:t]

# B is t x d orthogonal basis vectors, project it out of vector v
def subspace_project(B, v):
    for i in xrange(len(B)):
        v = v - np.dot(v, B[i]) * B[i]
    return v

# scale covariance down
def scaled_normal(d):
    return np.random.normal(scale = 1.0 / np.sqrt(d), size = d)

def top_vector_samples(U, n3, t):
    d = len(U)
    A1 = np.random.normal(scale = 1.0, size = (n3, d))
    diagonal = np.zeros(d)
    diagonal[:(t/2)] = np.sqrt(d) / np.sqrt(t)
    diagonal[(t/2):t] = 1.0
    basis_aligned = np.dot(U.T, np.dot(np.diag(diagonal), U))
    return np.dot(A1, basis_aligned)

# random n1 rows outside small subspace
# basis of subspace, blown up to be quite large
# other n2 random rows that are also large
def worse_instance(d, t, n1, n2, n3):
    U = ortho_group.rvs(d)
    B = U[:t]
    A1 = np.sqrt(2 * d) * top_vector_samples(U, n1, t)
    A2 = np.asarray([subspace_project(B, scaled_normal(d)) for i in xrange(n2)]) # should be unit norm?
    A3 = np.asarray([subspace_project(B, scaled_normal(d)) for i in xrange(n3)]) * d # should be norm about d
    A = np.vstack((A1, A2, A3))
    # np.random.shuffle(A)
    return A

def normalize(A):
    (n, d) = np.shape(A)
    B = np.zeros((n, d))
    for i in xrange(n):
        B[i] = A[i] / np.linalg.norm(A[i])
    return B

# Tau decrease: hard instance

In [ ]:
A = worse_instance(400, 4, 16, 6000, 200)

In [ ]:
degree = 10 # degree of rational approximation
jldirs = 7 # number of JL directions used
niters = 25 # number of iterations of solver
alpha = 1.0 # step size of solver

coeffs = np.asarray(ratapprox(degree))
reg = lin.Ridge(alpha=0.0, solver='lsqr', tol=1e-5)

In [ ]:
A1 = worse_instance(400, 4, 16, 6000, 200)
(w1, mvs1, taulist1) = mpc_ideal_niters_savetau(A1, 2000, alpha, niters, jldirs, coeffs)
print len(taulist1) # if terminated early, make sure to change the plot length

In [ ]:
A2 = worse_instance(400, 4, 16, 6000, 200)
(w2, mvs2, taulist2) = mpc_ideal_niters_savetau(A2, 2000, alpha, niters, jldirs, coeffs)
print len(taulist2)

In [ ]:
A3 = worse_instance(400, 4, 16, 6000, 200)
(w3, mvs3, taulist3) = mpc_ideal_niters_savetau(A3, 2000, alpha, niters, jldirs, coeffs)
print len(taulist3)

In [ ]:
plt.figure()
plt.errorbar(x=np.arange(0, 26, 1), y=taulist1)
plt.xticks(np.arange(0, 26, 5))
plt.title("Weighted tau values (run 1)")
plt.xlabel("Iteration number")
plt.ylabel("Weighted tau")
plt.show()

In [ ]:
plt.figure()
plt.errorbar(x=np.arange(0, 26, 1), y=taulist2)
plt.xticks(np.arange(0, 26, 5))
plt.title("Weighted tau values (run 2)")
plt.xlabel("Iteration number")
plt.ylabel("Weighted tau")
plt.show()

In [ ]:
plt.figure()
plt.errorbar(x=np.arange(0, 26, 1), y=taulist3)
plt.xticks(np.arange(0, 26, 5))
plt.title("Weighted tau values (run 3)")
plt.xlabel("Iteration number")
plt.ylabel("Weighted tau")
plt.show()

# Decrease l2 norm squared with respect to d

In [ ]:
def row_norm_noise(A, xtrue, stdev):
    (n, d) = np.shape(A)
    row_norms = np.asarray([np.linalg.norm(A[i]) for i in xrange(n)])
    noise = np.dot(np.diag(row_norms), np.random.normal(scale = stdev, size = n))
    return np.dot(A, xtrue) + noise

In [ ]:
# MSE for naive fits
squared_errors_dexp = np.zeros((10, 15))
for i in xrange(1, 11, 1):
    print i
    d = 50 * i
    (t, n1, n2, n3) = (4, 16, 10 * d, d / 2)
    A = worse_instance(d, t, n1, n2, n3)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(A, b).coef_
        squared_errors_dexp[i - 1][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
degree = 10 # degree of rational approximation
jldirs = 7 # number of JL directions used
niters = 20 # number of iterations of solver
alpha = 1.0 # step size of solver

coeffs = np.asarray(ratapprox(degree))
reg = lin.Ridge(alpha=0.0, solver='lsqr', tol=1e-5)

In [ ]:
# MSE for MPC fits
squared_errors_mpc_dexp = np.zeros((10, 15))
for i in xrange(1, 11, 1):
    print i
    d = 50 * i
    (t, n1, n2, n3) = (4, 16, 10 * d, d / 2)
    A = worse_instance(d, t, n1, n2, n3)
    (w, total_mv) = mpc_ideal_niters(A, 5 * d, alpha, niters, jldirs, coeffs)
    for j in xrange(15):
        xtrue = np.random.normal(size = d)
        b = row_norm_noise(A, xtrue, 0.1)
        x = reg.fit(WhalfA(A, w), np.sqrt(w) * b).coef_
        squared_errors_mpc_dexp[i - 1][j] = (np.linalg.norm(xtrue - x) ** 2)

In [ ]:
means_squared_errors_dexp = np.asarray([np.mean(squared_errors_dexp[i]) for i in xrange(10)])
means_squared_errors_dexp_mpc = np.asarray([np.mean(squared_errors_mpc_dexp[i]) for i in xrange(10)])
stdevs_squared_errors_dexp = np.asarray([np.std(squared_errors_dexp[i]) for i in xrange(10)])
stdevs_squared_errors_dexp_mpc = np.asarray([np.std(squared_errors_mpc_dexp[i]) for i in xrange(10)])

In [ ]:
plt.figure()
plt.errorbar(x=np.arange(50, 550, 50), y=means_squared_errors_dexp, yerr=stdevs_squared_errors_dexp)
plt.errorbar(x=np.arange(50, 550, 50), y=means_squared_errors_dexp_mpc, yerr=stdevs_squared_errors_dexp_mpc)
plt.xticks(np.arange(50, 550, 50))
plt.title("Mean squared errors, changing d")
plt.xlabel("Dimension d")
plt.ylabel("Mean squared error")
plt.show()